# OD Backup controller Align
At some point the rotatation arm can get out of zero and will need to be adjusted.  This helps you do that process.


To align (Servo version) follow the instruction in this notebook.

- Test alignment
- Make sure coarse alignment is ok
- Fine tune
- Test pickup and drop

## CDBurnerXP Driver Controller

In [1]:
import subprocess


class ODMedia:
    def __init__(self):
        self.path_root = r"C:\Program Files (x86)\CDBurnerXP\cdbxpcmd.exe"
    
    def version(self):
        result = subprocess.run([self.path_root, "--version"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def list_drives(self):
        result = subprocess.run([self.path_root, "--list_drives"], stdout=subprocess.PIPE
        )
        a = result.stdout.decode('utf-8').split('\n')
        print(f"|{a}|")
        return result.stdout.decode('utf-8')

    @property
    def number_of_drives(self):
        result = self.list_drives()
        if result.find('\n') == -1:
            return 1  # Todo check for 0
        else:
            return len(result.split('\n'))

    def disk_open(self):
        result = subprocess.run([self.path_root, "--eject", "--drivename:0"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def disk_close(self):
        result = subprocess.run([self.path_root, "--load", "--drivename:0"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

media = ODMedia()

## Robot Control
This imports the code.

In [2]:
import time

import od_control

robot = od_control.ODRobot(media.disk_open, media.disk_close, com_port="COM6")

Robot>import control1
>>> 


## Now start the list of commands to align the rotator

In [3]:
# Turn on robot if not on (making sure the drive can safely retract)
if media.number_of_drives == 1:
    print('Found D drive all ok')
else:
    raise Exception(f"Didn't find D drive, num = {media.number_of_drives}")

|['0: PIONEER  BD-RW   BDR-S09  (D:\\)\r', '']|
|['0: PIONEER  BD-RW   BDR-S09  (D:\\)\r', '']|


Exception: Didn't find D drive, num = 2

In [4]:
robot.zero_toolhead()  # Make sure toolhead out of the way

In [5]:
# Test the DVD
robot.disk_open()
robot.rotator.to_bin('od')
robot.move_toolhead_to(30)

## Check Coarse alignment
The toolhead should be above the CD drive. Put in a test CD to check alignment.
**If the toolhead is not close (Just beside hole) then**:
- Turn power off
- Rotate head by hand 
- Power on *Drive will be closed*
- Restart kernel and clear outputs
- Restart this notebook

## Fine tune alignement
Adjust defaults in code (TODO store in device)

## Test pickup and drop on Optical Drive

In [6]:
# Check grip, release, park and shutdown work
robot.grip()
robot.release()
robot.park()
robot.shutdown()

Robot>control1.th.grip()
>>> 
Robot>control1.th.release()
>>> 
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 


In [ ]:
robot.pickup_from_bin('od')
# If this does not happen smoothly then a fine adjustment of set posisition could help

In [ ]:
robot.drop_on_bin('od')
# Disk should be centered in drive

## Test pickup and drop on In Bin
First test alignment
Then check pickup and drop

In [ ]:
robot.pickup_from_bin('in')

In [ ]:
robot.drop_on_bin('in')

## Test pickup and drop on Out Bin
First test alignment
Then check pickup and drop.  Make sure have disck in out bin

In [ ]:
robot.pickup_from_bin('out')

In [ ]:
robot.drop_on_bin('out')

In [ ]:
robot.drop_on_out_bin()

## Testing waste bin
Make sure you have a test disk on the source.
Remember with cannot pickup from waste

In [ ]:
robot.pickup_from_bin('od')

In [ ]:
robot.drop_on_bin('waste')

## Test unloading from empty optical drive bin

In [ ]:
robot.unload()

In [ ]:
robot.load()

In [ ]:
robot.disk_close()

# Blank unload  test
The result was *Not safe*.  The toolhead went through the hole in the DVD drive and kept on going.  
I suppose you can test by using the DVD drive to check that the drive has a disk loaded before unloading.